In [1]:
import pandas as pd
import numpy as np

In [5]:
train=pd.read_csv('train.csv',encoding="utf-8-sig")             # feather 파일로 load 시 기존 CSV 파일 보다 2배 빠른 32초를 소요하여 로드 완료
test=pd.read_csv('test.csv',encoding="utf-8-sig")         

In [2]:
submission=pd.read_csv('sample_submission.csv',encoding="utf-8-sig")     

In [3]:
submission

,game_id,winner
0,38872,0
1,38873,0
2,38874,0
3,38875,0
4,38876,0
...,...,...
16782,55654,0
16783,55655,0
16784,55656,0
16785,55657,0


In [6]:
train.shape

(67091776, 7)

In [7]:
train.tail()

,game_id,winner,time,player,species,event,event_contents
67091771,38871,0,8.51,0,Z,Camera,"at (139.578125, 62.58203125)"
67091772,38871,0,8.52,1,T,GetControlGroup,NaN
67091773,38871,0,8.52,0,Z,Camera,"at (122.42578125, 45.4296875)"
67091774,38871,0,8.52,0,Z,Camera,"at (122.42578125, 43.25390625)"
67091775,38871,0,8.52,1,T,Ability,(1360) - TrainSCV


In [8]:
train.head()

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"


In [10]:
new_train = train[train["event"]!="Camera"]
new_test = test[test["event"]!="Camera"] 

In [80]:
df_train = new_train[new_train["game_id"] == 0]
#df_test = new_test[new_test["game_id"] == 0]

In [81]:
df_train = df_train[df_train["event"] == "Ability"]

In [82]:
df_train

,game_id,winner,time,player,species,event,event_contents
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
19,0,1,0.27,0,T,Ability,"(1021) - BuildSupplyDepot; Location: (135.0, 4..."
27,0,1,0.31,0,T,Ability,(480) - Stop
29,0,1,0.33,1,T,Ability,(1360) - TrainSCV
37,0,1,0.35,0,T,Ability,(1360) - TrainSCV
...,...,...,...,...,...,...,...
1090,0,1,7.11,0,T,Ability,"(1021) - BuildSupplyDepot; Location: (137.0, 5..."
1092,0,1,7.11,1,T,Ability,(1280) - LiftBarracks
1102,0,1,7.14,1,T,Ability,(1260) - BuildBarracksTechLab; Location: (36.5...
1117,0,1,7.18,1,T,Ability,(1300) - LiftStarport


In [71]:
import re

In [79]:
def fix_missing(df, col):
    for i, row in df.iterrows():
        val = row[col]
        split = val.event_contents.split("-")
        name = re.sub("[()]","",split[0])
        col_name = "player" + str(val.player) + "_" + name
        if col_name in df_train.columns:
            df_train[col_name] += 1
        else:
            df_train[col_name] = 1

In [83]:
fix_missing(df_train,["player","event_contents"])

In [89]:
df_train[-1:]

,game_id,winner,time,player,species,event,event_contents,player0_1360,player0_1021,player0_480,...,player0_1420,player0_12A0,player0_1446,player0_F61,player0_B40,player1_B40,player1_1280,player1_1260,player1_1300,player1_1380
1137,0,1,7.24,1,T,Ability,"(1380) - LandBarracks; Location: (28.5, 145.5,...",9,4,1,...,1,1,1,1,1,1,1,1,1,1


In [90]:
df_real_train = df_train[-1:]

In [95]:
df_real_train.reset_index(inplace=True)

In [96]:
df_real_train.drop(["index"],1,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [97]:
df_real_train

,game_id,winner,time,player,species,event,event_contents,player0_1360,player0_1021,player0_480,...,player0_1420,player0_12A0,player0_1446,player0_F61,player0_B40,player1_B40,player1_1280,player1_1260,player1_1300,player1_1380
0,0,1,7.24,1,T,Ability,"(1380) - LandBarracks; Location: (28.5, 145.5,...",9,4,1,...,1,1,1,1,1,1,1,1,1,1


In [98]:
df_real_train.to_csv("starcraft_train.csv",encoding="utf-8")